# Physics Tutorial 02: The Carnot Cycle

Visualizing why the Carnot engine is maximally efficient and why this cannot be exceeded.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, FancyBboxPatch
import matplotlib.patches as mpatches

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## Part 1: Carnot Efficiency Formula

$$\eta_{Carnot} = 1 - \frac{T_C}{T_H}$$

In [ ]:
def carnot_efficiency(T_H, T_C):
    """Calculate Carnot efficiency given hot and cold temperatures (in Kelvin)."""
    return 1 - T_C / T_H

# Example calculations
examples = [
    (373, 300, "Boiling water to room temp"),
    (500, 300, "Industrial heat to room temp"),
    (773, 300, "500°C to room temp"),
    (1273, 300, "1000°C to room temp"),
    (5778, 300, "Sun surface to Earth"),
]

print("Carnot Efficiency Examples:")
print("=" * 60)
print(f"{'Description':<30} | {'T_H (K)':<10} | {'T_C (K)':<10} | {'η':>6}")
print("-" * 60)

for T_H, T_C, desc in examples:
    eta = carnot_efficiency(T_H, T_C)
    print(f"{desc:<30} | {T_H:<10} | {T_C:<10} | {eta:>6.1%}")

In [ ]:
# Visualize efficiency vs temperatures
T_C = 300  # Room temperature
T_H_vals = np.linspace(301, 2000, 100)
efficiencies = [carnot_efficiency(T_H, T_C) for T_H in T_H_vals]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Efficiency vs T_H
axes[0].plot(T_H_vals, np.array(efficiencies) * 100, 'b-', linewidth=2)
axes[0].fill_between(T_H_vals, np.array(efficiencies) * 100, alpha=0.3)
axes[0].axhline(y=100, color='red', linestyle='--', linewidth=2, label='Theoretical max (100%)')
axes[0].axhline(y=50, color='gray', linestyle=':', alpha=0.5)

# Mark common operating points
markers = [(373, 'Boiling H₂O'), (773, '500°C'), (1273, '1000°C')]
for T, label in markers:
    eta = carnot_efficiency(T, T_C) * 100
    axes[0].plot(T, eta, 'ro', markersize=8)
    axes[0].annotate(f'{label}\n{eta:.1f}%', xy=(T, eta), xytext=(T+50, eta+5), fontsize=9)

axes[0].set_xlabel('Hot Temperature T_H (K)', fontsize=12)
axes[0].set_ylabel('Carnot Efficiency (%)', fontsize=12)
axes[0].set_title(f'Carnot Efficiency vs Hot Temperature\n(Cold reservoir at T_C = {T_C}K)', fontsize=14)
axes[0].legend(fontsize=10)
axes[0].set_ylim(0, 105)

# Efficiency vs T_C (for fixed T_H)
T_H = 773  # 500°C
T_C_vals = np.linspace(100, 772, 100)
efficiencies_2 = [carnot_efficiency(T_H, T_C) for T_C in T_C_vals]

axes[1].plot(T_C_vals, np.array(efficiencies_2) * 100, 'g-', linewidth=2)
axes[1].fill_between(T_C_vals, np.array(efficiencies_2) * 100, alpha=0.3, color='green')
axes[1].axhline(y=100, color='red', linestyle='--', linewidth=2, label='Theoretical max')

# Mark common cold temperatures
markers_c = [(300, 'Room temp'), (273, 'Ice point'), (200, 'Dry ice')]
for T, label in markers_c:
    eta = carnot_efficiency(T_H, T) * 100
    axes[1].plot(T, eta, 'go', markersize=8)
    axes[1].annotate(f'{label}\n{eta:.1f}%', xy=(T, eta), xytext=(T+30, eta-8), fontsize=9)

axes[1].set_xlabel('Cold Temperature T_C (K)', fontsize=12)
axes[1].set_ylabel('Carnot Efficiency (%)', fontsize=12)
axes[1].set_title(f'Carnot Efficiency vs Cold Temperature\n(Hot reservoir at T_H = {T_H}K = 500°C)', fontsize=14)
axes[1].legend(fontsize=10)
axes[1].set_ylim(0, 105)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  • Higher T_H → Higher efficiency")
print("  • Lower T_C → Higher efficiency")
print("  • 100% requires T_C = 0 (absolute zero) - impossible!")

## Part 2: The Carnot Cycle on a PV Diagram

In [ ]:
def carnot_cycle_points(T_H, T_C, V1, V2, n=1, R=8.314):
    """
    Calculate the four corner points of a Carnot cycle.
    
    Point 1 → 2: Isothermal expansion at T_H
    Point 2 → 3: Adiabatic expansion T_H → T_C
    Point 3 → 4: Isothermal compression at T_C
    Point 4 → 1: Adiabatic compression T_C → T_H
    """
    gamma = 5/3  # For monatomic ideal gas
    
    # Point 1
    P1 = n * R * T_H / V1
    
    # Point 2 (after isothermal expansion at T_H)
    P2 = n * R * T_H / V2
    
    # Point 3 (after adiabatic expansion to T_C)
    # For adiabatic: T*V^(gamma-1) = const
    V3 = V2 * (T_H / T_C) ** (1/(gamma-1))
    P3 = n * R * T_C / V3
    
    # Point 4 (after isothermal compression at T_C)
    V4 = V1 * (T_H / T_C) ** (1/(gamma-1))
    P4 = n * R * T_C / V4
    
    return {
        'V': [V1, V2, V3, V4],
        'P': [P1, P2, P3, P4],
        'T': [T_H, T_H, T_C, T_C]
    }

def isothermal_curve(T, V_range, n=1, R=8.314):
    """P = nRT/V for isothermal process."""
    return n * R * T / V_range

def adiabatic_curve(P_start, V_start, V_range, gamma=5/3):
    """PV^gamma = const for adiabatic process."""
    const = P_start * V_start ** gamma
    return const / V_range ** gamma

# Define the cycle
T_H = 600  # K
T_C = 300  # K
V1 = 0.001  # m³
V2 = 0.003  # m³

points = carnot_cycle_points(T_H, T_C, V1, V2)
V = points['V']
P = points['P']

# Create smooth curves for plotting
V_12 = np.linspace(V[0], V[1], 50)
P_12 = isothermal_curve(T_H, V_12)

V_23 = np.linspace(V[1], V[2], 50)
P_23 = adiabatic_curve(P[1], V[1], V_23)

V_34 = np.linspace(V[2], V[3], 50)
P_34 = isothermal_curve(T_C, V_34)

V_41 = np.linspace(V[3], V[0], 50)
P_41 = adiabatic_curve(P[3], V[3], V_41)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the four processes
ax.plot(V_12 * 1000, P_12 / 1000, 'r-', linewidth=2.5, label='1→2: Isothermal expansion (T_H)')
ax.plot(V_23 * 1000, P_23 / 1000, 'b-', linewidth=2.5, label='2→3: Adiabatic expansion')
ax.plot(V_34 * 1000, P_34 / 1000, 'c-', linewidth=2.5, label='3→4: Isothermal compression (T_C)')
ax.plot(V_41 * 1000, P_41 / 1000, 'm-', linewidth=2.5, label='4→1: Adiabatic compression')

# Mark the corner points
for i, (v, p) in enumerate(zip(V, P)):
    ax.plot(v * 1000, p / 1000, 'ko', markersize=10)
    ax.annotate(f'  {i+1}', xy=(v * 1000, p / 1000), fontsize=14, fontweight='bold')

# Shade the enclosed area (work done)
V_cycle = np.concatenate([V_12, V_23, V_34, V_41])
P_cycle = np.concatenate([P_12, P_23, P_34, P_41])
ax.fill(V_cycle * 1000, P_cycle / 1000, alpha=0.2, color='yellow', label='Net work output')

ax.set_xlabel('Volume (L)', fontsize=12)
ax.set_ylabel('Pressure (kPa)', fontsize=12)
ax.set_title(f'Carnot Cycle PV Diagram\nT_H = {T_H}K, T_C = {T_C}K, η = {carnot_efficiency(T_H, T_C):.1%}', fontsize=14)
ax.legend(loc='upper right', fontsize=10)

# Add arrows to show direction
ax.annotate('', xy=(V_12[25]*1000, P_12[25]/1000), xytext=(V_12[20]*1000, P_12[20]/1000),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))

plt.tight_layout()
plt.show()

print(f"\nCarnot cycle between T_H = {T_H}K and T_C = {T_C}K")
print(f"Efficiency: η = 1 - {T_C}/{T_H} = {carnot_efficiency(T_H, T_C):.1%}")

## Part 3: Why Carnot Cannot Be Exceeded — The Proof

In [ ]:
# Demonstrate the proof by contradiction
def analyze_combined_engines(eta_super, eta_carnot, W, T_H, T_C):
    """
    Analyze what happens when we combine a 'super-engine' with a reversed Carnot engine.
    """
    # Super-engine running forward
    Q_H_super = W / eta_super  # Heat absorbed from hot reservoir
    Q_C_super = Q_H_super - W  # Heat rejected to cold reservoir
    
    # Carnot engine running backward (as refrigerator)
    # It uses work W to pump heat
    Q_H_carnot = W / eta_carnot  # Heat delivered to hot reservoir
    Q_C_carnot = Q_H_carnot - W  # Heat absorbed from cold reservoir
    
    # Net effect on reservoirs
    net_hot = -Q_H_super + Q_H_carnot  # + means reservoir gains heat
    net_cold = Q_C_super - Q_C_carnot  # + means reservoir gains heat
    net_work = W - W  # = 0
    
    return {
        'super_Q_H': Q_H_super,
        'super_Q_C': Q_C_super,
        'carnot_Q_H': Q_H_carnot,
        'carnot_Q_C': Q_C_carnot,
        'net_hot': net_hot,
        'net_cold': net_cold,
        'net_work': net_work
    }

# Parameters
T_H = 600  # K
T_C = 300  # K
eta_carnot = carnot_efficiency(T_H, T_C)  # 50%
W = 100  # J of work per cycle

# What if a 'super-engine' had 60% efficiency?
eta_super = 0.60

result = analyze_combined_engines(eta_super, eta_carnot, W, T_H, T_C)

print("Proof that Carnot Efficiency Cannot Be Exceeded")
print("=" * 60)
print(f"\nSetup: T_H = {T_H}K, T_C = {T_C}K, W = {W}J")
print(f"Carnot efficiency: {eta_carnot:.0%}")
print(f"Hypothetical super-engine efficiency: {eta_super:.0%}")

print(f"\n--- Super-Engine (forward) ---")
print(f"Heat from hot reservoir: Q_H = {result['super_Q_H']:.1f} J")
print(f"Work produced: W = {W} J")
print(f"Heat to cold reservoir: Q_C = {result['super_Q_C']:.1f} J")

print(f"\n--- Carnot Engine (reversed, as refrigerator) ---")
print(f"Work consumed: W = {W} J")
print(f"Heat from cold reservoir: Q_C = {result['carnot_Q_C']:.1f} J")
print(f"Heat to hot reservoir: Q_H = {result['carnot_Q_H']:.1f} J")

print(f"\n--- NET EFFECT (combined) ---")
print(f"Net work: {result['net_work']:.1f} J (zero!)")
print(f"Net heat to hot reservoir: {result['net_hot']:.1f} J")
print(f"Net heat to cold reservoir: {result['net_cold']:.1f} J")

if result['net_hot'] > 0:
    print(f"\n>>> VIOLATION! <<<")
    print(f"Heat flows from COLD to HOT with NO work input!")
    print(f"This violates the Second Law of Thermodynamics!")
    print(f"\nConclusion: The super-engine CANNOT exist.")

In [ ]:
# Visualize the proof
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

def draw_engine_diagram(ax, title, Q_H, W, Q_C, direction='forward'):
    """Draw a heat engine energy flow diagram."""
    ax.set_xlim(-1, 3)
    ax.set_ylim(-1, 5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # Hot reservoir
    hot_rect = FancyBboxPatch((0.5, 4), 1, 0.6, boxstyle="round,pad=0.05", 
                               facecolor='red', edgecolor='black', alpha=0.7)
    ax.add_patch(hot_rect)
    ax.text(1, 4.3, f'HOT\n{T_H}K', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Engine
    engine_rect = FancyBboxPatch((0.3, 2), 1.4, 1, boxstyle="round,pad=0.05",
                                  facecolor='yellow', edgecolor='black', alpha=0.7)
    ax.add_patch(engine_rect)
    ax.text(1, 2.5, 'ENGINE', ha='center', va='center', fontsize=10, fontweight='bold')
    
    # Cold reservoir
    cold_rect = FancyBboxPatch((0.5, 0), 1, 0.6, boxstyle="round,pad=0.05",
                                facecolor='blue', edgecolor='black', alpha=0.7)
    ax.add_patch(cold_rect)
    ax.text(1, 0.3, f'COLD\n{T_C}K', ha='center', va='center', fontsize=10, fontweight='bold', color='white')
    
    # Arrows and labels
    if direction == 'forward':
        # Q_H down
        ax.annotate('', xy=(1, 3.2), xytext=(1, 3.9),
                    arrowprops=dict(arrowstyle='->', color='red', lw=3))
        ax.text(1.6, 3.5, f'Q_H = {Q_H:.0f}J', fontsize=10, color='red')
        
        # Q_C down
        ax.annotate('', xy=(1, 0.7), xytext=(1, 1.9),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=3))
        ax.text(1.6, 1.3, f'Q_C = {Q_C:.0f}J', fontsize=10, color='blue')
        
        # Work out
        ax.annotate('', xy=(2.3, 2.5), xytext=(1.8, 2.5),
                    arrowprops=dict(arrowstyle='->', color='green', lw=3))
        ax.text(2.4, 2.5, f'W = {W:.0f}J', fontsize=10, color='green')
    else:  # reversed (refrigerator)
        # Q_H up
        ax.annotate('', xy=(1, 3.9), xytext=(1, 3.2),
                    arrowprops=dict(arrowstyle='->', color='red', lw=3))
        ax.text(1.6, 3.5, f'Q_H = {Q_H:.0f}J', fontsize=10, color='red')
        
        # Q_C up
        ax.annotate('', xy=(1, 1.9), xytext=(1, 0.7),
                    arrowprops=dict(arrowstyle='->', color='blue', lw=3))
        ax.text(1.6, 1.3, f'Q_C = {Q_C:.0f}J', fontsize=10, color='blue')
        
        # Work in
        ax.annotate('', xy=(1.8, 2.5), xytext=(2.3, 2.5),
                    arrowprops=dict(arrowstyle='->', color='green', lw=3))
        ax.text(2.4, 2.5, f'W = {W:.0f}J', fontsize=10, color='green')

# Draw three diagrams
draw_engine_diagram(axes[0], 'Super-Engine (η=60%)', 
                    result['super_Q_H'], W, result['super_Q_C'], 'forward')

draw_engine_diagram(axes[1], 'Carnot Refrigerator', 
                    result['carnot_Q_H'], W, result['carnot_Q_C'], 'reversed')

# Combined effect
ax = axes[2]
ax.set_xlim(-1, 3)
ax.set_ylim(-1, 5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Combined Effect\n(IMPOSSIBLE!)', fontsize=12, fontweight='bold', color='red')

# Simplified diagram showing net heat flow
hot_rect = FancyBboxPatch((0.5, 3.5), 1, 0.8, boxstyle="round,pad=0.05", 
                           facecolor='red', edgecolor='black', alpha=0.7)
ax.add_patch(hot_rect)
ax.text(1, 3.9, f'HOT\n+{result["net_hot"]:.0f}J', ha='center', va='center', fontsize=10, fontweight='bold')

cold_rect = FancyBboxPatch((0.5, 0.5), 1, 0.8, boxstyle="round,pad=0.05",
                            facecolor='blue', edgecolor='black', alpha=0.7)
ax.add_patch(cold_rect)
ax.text(1, 0.9, f'COLD\n{result["net_cold"]:.0f}J', ha='center', va='center', fontsize=10, fontweight='bold', color='white')

# Arrow showing heat flow from cold to hot
ax.annotate('', xy=(1, 3.4), xytext=(1, 1.4),
            arrowprops=dict(arrowstyle='->', color='purple', lw=4))
ax.text(1.7, 2.4, f'Heat flows\nCOLD→HOT\nNo work!', fontsize=11, color='purple', fontweight='bold')

ax.text(1, -0.5, '⚠️ VIOLATES 2nd LAW!', ha='center', fontsize=12, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

## Part 4: Entropy Analysis

In [ ]:
# Show that exceeding Carnot would decrease total entropy
def entropy_analysis(Q_H, Q_C, T_H, T_C):
    """Calculate entropy changes for hot reservoir, cold reservoir, and total."""
    dS_hot = -Q_H / T_H  # Hot reservoir loses heat
    dS_cold = Q_C / T_C  # Cold reservoir gains heat
    dS_total = dS_hot + dS_cold
    return dS_hot, dS_cold, dS_total

# For various engine efficiencies
T_H = 600  # K
T_C = 300  # K
Q_H = 100  # J input

eta_carnot = carnot_efficiency(T_H, T_C)

print("Entropy Analysis for Different Engine Efficiencies")
print("=" * 70)
print(f"T_H = {T_H}K, T_C = {T_C}K, Q_H = {Q_H}J")
print(f"Carnot efficiency = {eta_carnot:.0%}")
print(f"\n{'Efficiency':>12} | {'Q_C':>8} | {'ΔS_hot':>10} | {'ΔS_cold':>10} | {'ΔS_total':>10} | {'Valid?':>8}")
print("-" * 70)

for eta in [0.30, 0.40, 0.50, 0.60, 0.70, 0.80]:
    W = eta * Q_H
    Q_C = Q_H - W
    dS_hot, dS_cold, dS_total = entropy_analysis(Q_H, Q_C, T_H, T_C)
    
    valid = "✓" if dS_total >= -1e-10 else "✗ VIOLATES!"
    carnot_marker = " (Carnot)" if abs(eta - eta_carnot) < 0.001 else ""
    
    print(f"{eta:>10.0%}{carnot_marker:>2} | {Q_C:>8.1f} | {dS_hot:>10.4f} | {dS_cold:>10.4f} | {dS_total:>10.4f} | {valid:>8}")

print("\nKey insight: Only η ≤ η_Carnot gives ΔS_total ≥ 0")
print("Exceeding Carnot efficiency would DECREASE total entropy - impossible!")

In [ ]:
# Visualize entropy change vs efficiency
efficiencies = np.linspace(0.01, 0.99, 100)
Q_H = 100

dS_totals = []
for eta in efficiencies:
    Q_C = Q_H * (1 - eta)
    _, _, dS_total = entropy_analysis(Q_H, Q_C, T_H, T_C)
    dS_totals.append(dS_total)

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(efficiencies * 100, dS_totals, 'b-', linewidth=2)
ax.axhline(y=0, color='red', linestyle='--', linewidth=2, label='ΔS = 0 (2nd Law boundary)')
ax.axvline(x=eta_carnot * 100, color='green', linestyle=':', linewidth=2, label=f'Carnot η = {eta_carnot:.0%}')

# Shade regions
ax.fill_between(efficiencies * 100, dS_totals, 0, 
                where=np.array(dS_totals) >= 0, 
                alpha=0.3, color='green', label='Allowed (ΔS ≥ 0)')
ax.fill_between(efficiencies * 100, dS_totals, 0, 
                where=np.array(dS_totals) < 0, 
                alpha=0.3, color='red', label='Forbidden (ΔS < 0)')

ax.set_xlabel('Engine Efficiency (%)', fontsize=12)
ax.set_ylabel('Total Entropy Change ΔS (J/K)', fontsize=12)
ax.set_title(f'Why Carnot is Maximum: Entropy Constraint\n(T_H={T_H}K, T_C={T_C}K)', fontsize=14)
ax.legend(fontsize=10, loc='upper right')

ax.set_xlim(0, 100)

plt.tight_layout()
plt.show()

print(f"\nThe Second Law (ΔS ≥ 0) directly implies η ≤ η_Carnot")
print(f"This is why Carnot efficiency cannot be exceeded!")

## Part 5: Real vs Ideal Engines

In [ ]:
# Compare real engine efficiencies to their Carnot limits
engines = [
    ("Car engine (gasoline)", 600, 300, 0.25),
    ("Diesel engine", 700, 300, 0.35),
    ("Coal power plant", 800, 300, 0.38),
    ("Nuclear power plant", 600, 300, 0.33),
    ("Combined cycle gas turbine", 1400, 300, 0.58),
    ("Fuel cell (theoretical)", 1200, 300, 0.65),
]

print("Real Engines vs Carnot Efficiency")
print("=" * 80)
print(f"{'Engine Type':<30} | {'T_H (K)':<8} | {'η_actual':>10} | {'η_Carnot':>10} | {'% of Carnot':>12}")
print("-" * 80)

engine_data = []
for name, T_H, T_C, eta_actual in engines:
    eta_c = carnot_efficiency(T_H, T_C)
    pct_carnot = eta_actual / eta_c
    print(f"{name:<30} | {T_H:<8} | {eta_actual:>10.0%} | {eta_c:>10.1%} | {pct_carnot:>12.0%}")
    engine_data.append((name, eta_actual * 100, eta_c * 100, pct_carnot * 100))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

names = [d[0] for d in engine_data]
actual = [d[1] for d in engine_data]
carnot = [d[2] for d in engine_data]

x = np.arange(len(names))
width = 0.35

bars1 = ax.bar(x - width/2, actual, width, label='Actual Efficiency', color='steelblue')
bars2 = ax.bar(x + width/2, carnot, width, label='Carnot Limit', color='coral', alpha=0.7)

ax.set_xlabel('Engine Type', fontsize=12)
ax.set_ylabel('Efficiency (%)', fontsize=12)
ax.set_title('Real Engine Efficiency vs Carnot Theoretical Limit', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=30, ha='right', fontsize=10)
ax.legend(fontsize=11)
ax.set_ylim(0, 100)

# Add percentage labels
for i, (a, c) in enumerate(zip(actual, carnot)):
    ax.text(i - width/2, a + 2, f'{a:.0f}%', ha='center', fontsize=9)
    ax.text(i + width/2, c + 2, f'{c:.0f}%', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nKey insight: Real engines achieve 50-80% of their Carnot limit.")
print("The gap is due to friction, heat leaks, finite-time processes, etc.")

## Summary

**Why Carnot efficiency is the maximum:**

1. **Mathematical proof**: A super-engine + reversed Carnot = heat pump with no work
   - This violates the Second Law (heat can't flow cold→hot spontaneously)
   - Therefore, no super-engine can exist

2. **Entropy proof**: For any engine:
   - $\Delta S_{total} = Q_C/T_C - Q_H/T_H \geq 0$ (Second Law)
   - This directly implies $\eta \leq 1 - T_C/T_H = \eta_{Carnot}$

3. **Physical insight**:
   - Carnot cycle is reversible (no entropy generated)
   - Any irreversibility wastes energy as extra heat
   - Maximum efficiency = minimum waste = reversible process

**The Carnot efficiency $\eta = 1 - T_C/T_H$ is not just an engineering limit — it's a law of nature.**